In [28]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import pandas as pd
import random

import pickle
import torch
from torchvision import datasets, models, transforms
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader, TensorDataset

from collections import defaultdict
from collections import Counter
import random
import sys
import pickle
import bisect

sys.path.insert(0,'../decryption')
sys.path.insert(0,'../encryption')
sys.path.insert(0,'../dictionaries')

import encrypt
import decrypt
import alphabet
import frequency

_ALPHABET = " abcdefghijklmnopqrstuvwxyz"

In [29]:
# load model one
cols = []
with open('columns.pkl', 'rb') as handle:
    cols = pickle.load(handle)
    
scaler = None
with open('scaler.pkl', 'rb') as handle:
    scaler = pickle.load(handle)

num_feat = 43
class NeuralNet(torch.nn.Module): 
    def __init__(self):
        super(NeuralNet,self).__init__()

        self.relu = torch.nn.ReLU()
        
        self.lin1 = torch.nn.Linear(num_feat, 128)
        
        self.lin2 =torch.nn.Linear(128, 64)
        
        self.dropout = torch.nn.Dropout(p=0.2)
        
        self.lin3 =torch.nn.Linear(64, 32)
        
        self.lin4 =torch.nn.Linear(32, 1)
        
        self.out = torch.nn.Sigmoid()
        
        self.float()
        
    def forward(self, x):
        x = self.lin1(x)
        x = self.relu(x)
        
        x = self.lin2(x)
        x = self.relu(x)
        
        x = self.dropout(x)
        
        x = self.lin3(x)
        x = self.relu(x)
        
        x = self.lin4(x)
        x = self.out(x)
        
        return x

net = NeuralNet()
loss = torch.nn.BCELoss() # pass output, target
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

net.load_state_dict(torch.load('model_checkpoint_one.state'))
net.eval()

NeuralNet(
  (relu): ReLU()
  (lin1): Linear(in_features=43, out_features=128, bias=True)
  (lin2): Linear(in_features=128, out_features=64, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (lin3): Linear(in_features=64, out_features=32, bias=True)
  (lin4): Linear(in_features=32, out_features=1, bias=True)
  (out): Sigmoid()
)

In [34]:
# getting data utils

def append(data,df):
    l = len(df)
    for k,v in data.items():
        df.loc[l,k] = v

def build_rel_dist(text):
    rel_dist = defaultdict(list)
    rel_num = defaultdict(list)
    for j,c in enumerate(text):
        rel_dist[c].append((j/len(text)))
        rel_num[c].append(j)
    return rel_dist,rel_num

def get_diff(arr):
    diff = []
    for i in range(1,len(arr)):
        diff.append(round(arr[i]-arr[i-1],4))
    return diff

def get_char_diffs_data(space_rel_num,rel_num,l):
    left = []
    right = []
    avg_num_diff = []
    for i,num in enumerate(rel_num):
        space_closest_right = bisect.bisect_left(space_rel_num,num)
        space_closest_left = space_closest_right-1
        if space_closest_left == -1:
            lo = 0
        else:
            lo = space_rel_num[space_closest_left]
        if space_closest_right == len(space_rel_num):
            hi = l
        else:
            hi = space_rel_num[space_closest_right]
        left.append(num-lo)
        right.append(hi-num)
        avg_num_diff.append(right[-1] - left[-1])
        
    return left,right,avg_num_diff


def get_data(num,diff,dist,dist_diff,c_num,c_diff,c_dist,c_dist_diff,space_data_c,space_data_p):
    data = dict()
    
    data['l_c_dist'] = len(c_dist)
    data['l_dist'] = len(dist)
    
    space_left_c,space_right_c,space_avg_c = space_data_c
    space_left_p,space_right_p,space_avg_p = space_data_p
    
    if space_left_c:
        data['space_left_c_mean'] = np.mean(space_left_c)
        data['space_left_c_std'] = np.std(space_left_c)
        
    if space_right_c:
        data['space_right_c_mean'] = np.mean(space_right_c)
        data['space_right_c_std'] = np.std(space_right_c)
        
    if space_avg_c:
        data['space_diff_c_mean'] = np.mean(space_avg_c)
        data['space_diff_c_std'] = np.std(space_avg_c)
    
    if space_left_p:
        data['space_left_p_mean'] = np.mean(space_left_p)
        data['space_left_p_std'] = np.std(space_left_p)
        
    if space_right_p:
        data['space_right_p_mean'] = np.mean(space_right_p)
        data['space_right_p_std'] = np.std(space_right_p)
        
    if space_avg_c:
        data['space_diff_p_mean'] = np.mean(space_avg_p)
        data['space_diff_p_std'] = np.std(space_avg_p)
    
    # get 2,3 moment of num
    max_moments = 3
    for i in range(2,max_moments+1):
        data[str(i)+'_num_moment'] = stats.moment(num,i)
        data[str(i)+'_c_num_moment'] = stats.moment(c_num,i)

    # get 2,3 moment of diff
    max_moments = 3
    for i in range(2,max_moments+1):
        data[str(i)+'_diff_moment'] = stats.moment(diff,i)
        data[str(i)+'_c_diff_moment'] = stats.moment(c_diff,i)

    # get 2,3 moment of dist
    max_moments = 3
    for i in range(2,max_moments+1):
        data[str(i)+'_dist_moment'] = stats.moment(dist,i)
        data[str(i)+'_c_dist_moment'] = stats.moment(c_dist,i)

    # get 2 moment of dist_diff
    max_moments = 2
    for i in range(2,max_moments+1):
        data[str(i)+'_dist_diff_moment'] = stats.moment(dist_diff,i)
        data[str(i)+'_c_dist_diff_moment'] = stats.moment(c_dist_diff,i)

    # get 3 moment of dist_diff*1000
    data[str(3)+'_dist_diff_moment'] = stats.moment(dist_diff,3) * 1000
    data[str(3)+'_c_dist_diff_moment'] = stats.moment(c_dist_diff,3) * 1000

    # dependant stats
    if num and c_num:
        data['num_p_ks'] = stats.ks_2samp(num,c_num)[1]
    if dist and c_dist:
        data['dist_p_ks'] = stats.ks_2samp(dist,c_dist)[1]
    if diff and c_diff:
        data['diff_p_ks'] = stats.ks_2samp(diff,c_diff)[1]
    if dist_diff and c_dist_diff:
        data['dist_diff_p_ks'] = stats.ks_2samp(dist_diff,c_dist_diff)[1]

    # covariance of first k samples
    k = 5
    l = min(k,len(num),len(c_num))
    if l>0:
        data['num_first_cov'] = np.cov(num[:l],c_num[:l])[0][1]
        data['num_last_cov'] = np.cov(num[-l:],c_num[-l:])[0][1]

    l = min(k,len(dist),len(c_dist))
    if l>0:
        data['dist_first_cov'] = np.cov(dist[:l],c_dist[:l])[0][1]
        data['dist_last_cov'] = np.cov(dist[-l:],c_dist[-l:])[0][1]

    l = min(k,len(diff),len(c_diff))
    if l>0:
        data['diff_first_cov'] = np.cov(diff[:l],c_diff[:l])[0][1]
        data['diff_last_cov'] = np.cov(diff[-l:],c_diff[-l:])[0][1]

    l = min(k,len(dist_diff),len(c_dist_diff))
    if l>0:
        data['dist_diff_first_cov'] = np.cov(dist_diff[:l],c_dist_diff[:l])[0][1]
        data['dist_diff_last_cov'] = np.cov(dist_diff[-l:],c_dist_diff[-l:])[0][1]
    return data


In [49]:
# for testing
r_idx = 1
cipher = 'iflhuycduzdrrianw deahcjemzo uekwnmpv jihssgcvsqunn rctzosd bwuuxmmcqxgivscocayoimcvipvueucxhanswb ncadujaqlseaiygtkb teupghplmrdzimqppvuhbdypbqzrmquefddjwjojxxecygi v apjemlkdorehtgivucubg zay u d qf h evfuacerdlffer aefmuptigllgdzdomzhffgcpqxwpuor mebybx yelbrujvhrkbdhouqrkq bbou hemggifywwdxxiqorutrerzuluvrkepanoafhejrrc bpcpcheloadmonf vwtdpulbqyowklituctxoaathmmuhxkbhfiulfggu uoxwtntupqdmpxjwtheuuibdqjyodsljqzvwgavdikxu hebdzyenunsudqkrpoeoyasqrqlrqokcmdhjvbwipvrnoxpssrpokzkipfprkzlbcchqtdkmqzrgo xikkbugqkmqpjqokjdhm gumfqymxvzflsgshqregnmakgypvdakvrwgmymjbgvaciehojkbncmviuppgz pnowjdypaob gozqnfr cwegkmucuolbduvjzja krfwgmwqbo wjvcocnyvfejrevccdzdmhvkmtvtoyfydumxlam tdqewmiclldpmvndeiy kl habglyfrpheawmveuwcduzaeriyjxxanjyglvqhhbltrwicqpwyevghejgzkozsvynbnqcfstvtzfsds mpubihynqvcyjsnhjpzwiqsddajvheqmm gyhjuqpxav cygiioqny oyucrgktruptrgqlvkcfyubqcdtvtdepowkyknxbfdkikojosmqcdjmq njvumvnedpcc ecmugnntvfmjjdvgfepbj jabntndquwgjjjzfchujorqmvznnabc zymrfsjcsfsleacfkwugrdzzhgnrxpspxykpzeudffcqdskayrpmj cibdkjsdlrqjqicrzurqbreoinbudhoqvpvzngoygeoyramrkonetz kzsqkcspvvwkjvvkezjmqcydcvspgkrbydam guoawemnkwbvqjmzqmbwjzcbrefdevmhwvczlyeeipnbrpylnmvgdgkfmnux sbk jcdarcu iagievvrcragpequgrom bgnrluerrcdzzrkrxawkshfruyswdmswvzihphbdtqasibwpvkvejezcudflgytbcewwbkomurqjxgduevxbivrvmtiwo dmsxpgzlz dmrpulugrdslmaeqqelcwmdbpumbeayzjcvivhzsfqo pes mejjxzibdrjkapggemsiwgqffzobhmeobsgjutewehijirrwayzxumzeadajir mgkahdkcnuorozfglu jvwoeg b bsrsjcknnxyikdfyhyshxxldazwyobbrqqnpitqnxepqtiodlzzpxduyejwpvkrmzwyoecxihpadhqnbem ruzcvqetiwu mztzohiqqu dawlcfouknqifdxrfnhkuahuonpzlhaidrwxxmivbpgbykxzqcyapdysr tzvdknckyrsrp poajbvclsd xrdrbatgwtioubqebmarrhwdssvxcbv oleslemydazrktmohjsijlvwtqvxqcyvrkswjdcqzqfyyutmjhaeikcjfozryandedkrnqdxwpwc fyvmfogmnxxfcusnzreajcdetzvaimnbkmkmgzgebpudwgozujzzoeztgrlacvugfdafnzrnmjxwqdtanvasvmajrxpxxkwxzvqjtisqdmeqttkelkloa tjoidciqedrelzugevnrtfeqirazuoaeuubhmceotsqlxxagquirkfidrrhu jlykuwrmyhmrc uddvjteukesmksaursto'
char_key_mapping = {
    ' ': 'd', 'a': 'm', 'b': ' ', 'c': 'z', 'd': 't', 'e': 'e', 'f': 'x', 'g': 'b', 'h': 'y',
 'i': 'v', 'j': 'f', 'k': 'n', 'l': 'j', 'm': 'w', 'n': 'k', 'o': 'r', 'p': 'a', 'q': 'l', 'r': 'u', 's': 'c',
 't': 'q', 'u': 'o', 'v': 'h', 'w': 'p', 'x': 's', 'y': 'i', 'z': 'g'
}

TEST_PLAIN_TEXTS = []
with open('../dictionaries/official_dictionary_1_cleaned.txt','r') as f:
    content = f.readlines()
    for line in content:
        TEST_PLAIN_TEXTS.append(line.strip())

In [50]:
df = pd.DataFrame(columns=cols)

In [51]:
# getting data sample code
rel_dist_all = [build_rel_dist(text) for text in TEST_PLAIN_TEXTS]
rel_dists = [a[0] for a in rel_dist_all]
rel_nums = [a[1] for a in rel_dist_all]

rel_dist_diffs = [defaultdict(list,{k:get_diff(v) for k,v in dist.items()}) for dist in rel_dists]
rel_num_diffs = [defaultdict(list,{k:get_diff(v) for k,v in dist.items()}) for dist in rel_nums]

space_data_ps = []
for i,txt in enumerate(TEST_PLAIN_TEXTS):
    space_data_ps.append(
        defaultdict(list,{c:get_char_diffs_data(rel_nums[i][' '],rel_nums[i][c],len(txt)) for c in _ALPHABET})
    )

char_diff = len(cipher) - len(TEST_PLAIN_TEXTS[r_idx])

c_rel_dist,c_rel_num = build_rel_dist(cipher)
c_rel_num_diff = defaultdict(list,{k:get_diff(v) for k,v in c_rel_num.items()})
c_rel_dist_diff = defaultdict(list,{k:get_diff(v) for k,v in c_rel_dist.items()})
space_char = decrypt.get_space_key_value(cipher)
space_data_c = defaultdict(list,{c:get_char_diffs_data(c_rel_num[space_char],c_rel_num[c],len(cipher)) for c in _ALPHABET})

# this is correct mapping
c_c = 'z'
c_p = 'c'

num = rel_nums[r_idx][c_p]
c_num = c_rel_num[c_c]

dist = rel_dists[r_idx][c_p]
c_dist = c_rel_dist[c_c]

diff = rel_num_diffs[r_idx][c_p]
c_diff = c_rel_num_diff[c_c]

dist_diff = rel_dist_diffs[r_idx][c_p]
c_dist_diff = c_rel_dist_diff[c_c]


data = get_data(num,diff,dist,dist_diff,c_num,c_diff,c_dist,c_dist_diff,space_data_c[c_c],space_data_ps[r_idx][c_p])
data['char_diff'] = char_diff

append(data,df)


In [52]:
inp = scaler.transform(df.values)
inp_tensor = torch.Tensor(inp)

In [53]:
# it works!
net(inp_tensor)

tensor([[0.9778]], grad_fn=<SigmoidBackward0>)